# Деревья решений

## Построение дерева

Опишем жадный алгоритм построения бинарного дерева решений:
1. Начинаем со всей обучающей выборки $X$, которую помещаем в корень $R_1$. 
2. Задаём функционал качества $Q(X, j, t)$ и критерий остановки. 
3. Запускаем построение из корня: $SplitNode(1, R_1)$

Функция $SplitNode(m, R_m)$
1. Если выполнен критерий остановки, то выход.
2. Находим наилучший с точки зрения $Q$ предикат: $j, t$: $[x_j<t]$
3. Помещаем предикат в вкршину и получаем с его помощью разбиение $X$ на две части: $R_{left} = \lbrace x|x_j<t \rbrace$ и $R_{right} = \lbrace x|x_j \geqslant t \rbrace$
4. Поместим $R_{left}$ и $R_{right}$ соответсвенно в левое и правое поддерево.
5. Рекурсивно повторяем $SplitNode(left, R_{left})$ и $SplitNode(right, R_{right})$.

В конце поставим в соответствие каждому листу ответ. Для задачи классификации - это самый частый среди объектов класс или вектор с долями классов (можно интерпретировать как вероятности):
$$ c_v = \arg \max_{k\in Y} \sum_{(x_i,y_i) \in R_v} [y_i=k]  $$

## Функционал качества для деревьев решений


Энтропия Шеннона для системы с N возможными состояниями определяется по формуле:
$$H = - \sum_{i=0}^{N} p_i\log_2p_i $$

где $p_i$ – вероятности нахождения системы в $i$-ом состоянии. 

Это очень важное понятие теории информации, которое позволяет оценить количество информации (степень хаоса в системе). Чем выше энтропия, тем менее упорядочена система и наоборот. С помощью энтропии мы формализуем функционал качества для разделение выборки (для задачи классификации).

In [16]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import random
from pprint import pprint

Код для расчёта энтропии:

In [17]:
def entropy(y):
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

Здесь $y$ - это массив значений целевой переменной

Энтропия – по сути степень хаоса (или неопределенности) в системе. Уменьшение энтропии называют приростом информации (information gain, IG).

Обочначим $R_v$ - объекты, которые нужно разделить в помощью предиката в вершине $v$. Запишем формулу для расчёта информационного прироста:
$$ Q = IG = H(R_v) - (H(R_{left})+H(R_{right}))$$

На каждом шаге нам нужно максимизировать этот функционал качества. Как это делать? Например, так можно перебрать $t$ для выбранного $j$.

Предыдущая версия формулы прироста информации слишком упрощена. В работе необходимо использовать более устойчивую формулу, которая учитывает не только энтропию подмножеств, но и их размер. 

$$ Q = IG = H(R_v) - \Big (\frac{|R_{left}|} {|R_{v}|} H(R_{left})+ \frac{|R_{right}|} {|R_{v}|} H(R_{right})\Big)$$

где, $|R_{v}|$, $|R_{left}|$ и $|R_{right}|$ - количество элементов в соответствующих множествах.

In [18]:
def information_gain(v, left, right):
    r_v = len(v)
    r_left = len(left)
    r_right = len(right)
    IG = entropy(v) - (r_left / r_v * entropy(left) + r_right / r_v * entropy(right))
    return IG


### Задание 4.1

Реализуйте алгоритм построения дерева. Должны быть отдельные функции (методы):

1.   для расчёта энтропии (уже есть)
2.   для разделения дерева (используйте pandas)
3.   для подсчёта функционала качества $IG$
4.   для выбора наилучшего разделения (с учетом признаков и порогов)~~
5.   для проверки критерия остановки

Для набора данных `iris` реализуйте алгоритм и минимум три из разных критериев остановки из перечисленных ниже:
* **максимальной глубины дерева = 5**
* **минимального числа объектов в листе = 5**
* максимальное количество листьев в дереве = 5
* **purity (остановка, если все объекты в листе относятся к одному классу)**

**Реализуйте** функцию `predict` (на вход функции подаётся датафрейм с объектами)

**Оцените точность** каждой модели с помощью метрики точность (`from sklearn.metrics import accuracy_score` или реализовать свою).

In [19]:
from scipy.stats import mode
from sklearn import datasets

In [20]:
class DecisionTree():
    def __init__(self, columns = datasets.load_iris().feature_names, max_depth=1000, min_leaf=1):
        self.columns = columns
        self.depth = 0
        self.max_depth = max_depth
        self.min_leaf = min_leaf
        self.tree = None

    def split(self, column, y):
        best_IG = -1
        best_t = None

        for value in set(column):
            y_predict = column < value
            IG = information_gain(y, y[y_predict], y[~y_predict])
            if IG > best_IG:
                best_t, best_IG = value, IG
        return best_t, best_IG

    def best_split(self, X, y):
        column = None
        best_t = None
        best_IG = -1

        for i, c in enumerate(X.T):
            t, IG = self.split(c, y)
            if IG > best_IG:
                best_t, best_IG = t, IG
                column = i
        return best_t, best_IG, column

    def criteria(self, criteria, depth):
        if criteria == 'max_depth':
            return depth >= self.max_depth
        elif criteria == 'min_leaf':
            return len(y) < self.min_leaf
        elif criteria == 'purity':
            return self.all_same(y)

    def fit(self, X, y, criteria, parent_node={}, depth=0):
        if len(y) == 0:
            return {}
        elif len(y) == 1:
            return {'class': y[0]}

        if self.criteria(criteria, depth):
            return None

        t, IG, column = self.best_split(X, y)

        y_left = y[X[:, column] < t]
        y_right = y[X[:, column] >= t]

        if len(y_left) == 0 or len(y_right) == 0:
            return {'class': mode(y).mode[0]}

        parent_node = {'column': self.columns[column], 
                       'column index': column, 't': t,
                       'class': mode(y).mode[0],
                       'left': self.fit(X[X[:, column] < t], y_left, criteria, {}, depth + 1)}

        if parent_node['left'] is None:
            parent_node = {'class': mode(y).mode[0]}
            return parent_node

        parent_node['right'] = self.fit(X[X[:, column] >= t], y_right, criteria, {}, depth + 1)

        if parent_node['right'] is None:
            parent_node = {'class': mode(y).mode[0]}
            return parent_node

        self.depth += 1
        self.tree = parent_node
        return parent_node

    def all_same(self, y):
        return all(x == y[0] for x in y)

    def predict(self, X):
        results = np.array([0] * len(X))
        for i, c in enumerate(X):
            results[i] = self._get_prediction(c)
        return results

    def _get_prediction(self, row):
      tree = self.tree
      while tree.get('t'):
          if row[tree['column index']] < tree['t']:
              tree = tree['left']
          else:
              tree = tree['right']
      else:
          return tree.get('class')


In [21]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

In [22]:
iris = datasets.load_iris()

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.33, random_state=42)

clf1 = DecisionTree()
model1 = clf1.fit(X_train, y_train, 'purity')

clf2 = DecisionTree(max_depth=5)
model2 = clf2.fit(X_train, y_train, 'max_depth')

clf3 = DecisionTree(min_leaf=5)
model3 = clf3.fit(X_train, y_train, 'min_leaf') 

In [23]:
y1_pred = clf1.predict(X_test)
y2_pred = clf2.predict(X_test)
y3_pred = clf3.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score

print("Accuracy score: purity ", accuracy_score(y_test, y1_pred))
print("Accuracy score: max_depth ", accuracy_score(y_test, y2_pred))
print("Accuracy score: min_leaf ", accuracy_score(y_test, y3_pred))

Accuracy score: purity  0.96
Accuracy score: max_depth  0.98
Accuracy score: min_leaf  0.96


In [25]:
from sklearn.tree import DecisionTreeClassifier

tree1 = DecisionTreeClassifier(criterion='entropy')
tree2 = DecisionTreeClassifier(criterion='entropy', max_depth = 5)
tree3 = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=5)

tree1.fit(X_train, y_train)
tree2.fit(X_train, y_train)
tree3.fit(X_train, y_train)

tree1_pred = tree1.predict(X_test)
tree2_pred = tree1.predict(X_test)
tree3_pred = tree2.predict(X_test)

print("Accuracy score sklearn: purity ", accuracy_score(y_test, tree1_pred))
print("Accuracy score sklearn: max_depth ", accuracy_score(y_test, tree2_pred))
print("Accuracy score sklearn: min_leaf ", accuracy_score(y_test, tree3_pred))

Accuracy score sklearn: purity  0.98
Accuracy score sklearn: max_depth  0.98
Accuracy score sklearn: min_leaf  0.96


##  Случайный лес

Опишем алгоритм случайный лес (*random forest*) и попутно разберём основные идеи:

1. Зададим $N$ - число деревьев в лесу.
2. Для каждого $n$ из $N$ сгенерируем свою выборку $X_n$. Пусть $m$ - это количество объектов в $X$. При генерации каждой $X_n$ мы будем брать объекты $m$ раз с возвращением. То есть один и тот же объект может попасть в выборку несколько раз, а какие-то объекты не попадут. (Этот способ назвается бутстрап).
3. По каждой $X_n$ построим решающее дерево $b_n$. Обычно стараются делать глубокие деревья. В качестве критериев остановки можно использовать `max_depth` или `min_samples_leaf` (например, пока в каждом листе не окажется по одному объекту). При каждом разбиении сначала выбирается $k$ (эвристика $k = \sqrt d$, где $d$ - это число признаков объектов из выборки $X$) случайных признаков из исходных, и оптимальное разделение выборки ищется только среди них. Обратите внимание, что мы не выбрасываем оставшиеся признаки!
4. Итоговый алгоритм будет представлять собой результат голосования (для классификации) и среднее арифметическое (для регрессии). Модификация алгоритма предполагает учёт весов каждого отдельного слабого алгоритма в ансамбле, но в этом особо нет смысла.


### Задание 4.2

В качестве набора данных используйте: https://www.kaggle.com/mathchi/churn-for-bank-customers

Там есть описание и примеры работы с этими данными. Если кратко, речь идёт про задачу прогнозирования оттока клиентов. Есть данные о 10 тысячах клиентов банка, часть из которых больше не являются клиентами.

Используя либо свою реализацию, либо  `DecisionTreeClassifier` с разными настройками из `sklearn.tree` реализйте алгоритм "случайный лес". 

Найдите наилучшие гиперпараметры этого алгоритма: количество деревьев, критерий остановки, функционал качества, минимальное количество объектов в листьях и другие.

Нельзя использовать готовую реализацию случайного леса из `sklearn`.

В подобных задачах очень важна интерпретируемость алгоритма. Попытайтесь оценить информативность признаков, т.е. ответить а вопрос, значения каких признаков являются самыми важными индикаторами того, что банк потеряет клиента.

In [26]:
from collections import Counter
from sklearn.base import BaseEstimator

In [27]:
class RandomForest(BaseEstimator):
    def __init__(self, n_trees=10, min_samples_split=2,
                max_depth=100, criterion='gini', max_leaf_nodes=20):
      self.n_trees = n_trees
      self.max_depth = max_depth
      self.min_samples_split = min_samples_split
      self.criterion = criterion
      self.max_leaf_nodes = max_leaf_nodes
      self.trees = []

    def bootstrap(self, X, y):
      n_samples = len(X)
      indexes = np.random.choice(n_samples, n_samples, replace=True)
      return X.iloc[indexes, :], y.iloc[indexes]

    def most_common(self, y):
      return Counter(y).most_common(1)[0][0]

    def fit(self, X, y):
      self.trees = []
      for i in range(self.n_trees):
          tree = DecisionTreeClassifier(max_depth=self.max_depth,
                                        min_samples_split=self.min_samples_split,
                                        criterion=self.criterion,
                                        max_leaf_nodes=self.max_leaf_nodes, max_features='sqrt')
          X_, y_ = self.bootstrap(X, y)
          tree.fit(X_, y_)
          self.trees.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.trees])
        predictions = np.swapaxes(predictions, 0, 1)
        y_predicion = [self.most_common(tree_pred) for tree_pred in predictions]
        return np.array(y_predicion)


Выполним обработку данных:

In [28]:
from google.colab import drive

In [29]:
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
churn = pd.read_csv('/content/drive/MyDrive/data/churn.csv', sep=',')

churn

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Выполним обработку датасета (оставим только необходимые столбцы):

In [31]:
churn = churn.drop(labels=['RowNumber', 'CustomerId', 'Surname'], axis=1)

churn

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


Заменим текстовое значение Gender на цифровые значения: 1 - Male, 0 - Female

In [32]:
churn.Gender.replace(['Male', 'Female'], [1, 0], inplace=True)

churn

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


Закодируем столбец Geography с помощью Target Encoding: 

In [33]:
!pip install category_encoders

     |████████████████████████████████| 82 kB 422 kB/s 


In [34]:
from category_encoders import TargetEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [35]:
encoder = TargetEncoder()
churn['Geography'] = encoder.fit_transform(churn['Geography'], churn['Exited'])

churn

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0.161548,0,42,2,0.00,1,1,1,101348.88,1
1,608,0.166734,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0.161548,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0.161548,0,39,1,0.00,2,0,0,93826.63,0
4,850,0.166734,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0.161548,1,39,5,0.00,2,1,0,96270.64,0
9996,516,0.161548,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,0.161548,0,36,7,0.00,1,0,1,42085.58,1
9998,772,0.324432,1,42,3,75075.31,2,1,0,92888.52,1


Присупим к построению случайного леса:

In [36]:
from sklearn.model_selection import GridSearchCV

In [42]:
X = churn.iloc[:,:-1]
y = churn.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = RandomForest()

param_grid = {
    'n_trees': np.arange(1, 101, 5),
    # 'min_samples_split': np.arange(2, 7),
    'max_depth': np.arange(5, 21, 3),
    'criterion': ['gini', 'entropy'],
    # 'max_leaf_nodes': np.arange(10, 51, 10)
}

search = GridSearchCV(clf, param_grid, scoring='accuracy')
search.fit(X_train, y_train)

print(search.best_params_)

{'criterion': 'gini', 'max_depth': 11, 'n_trees': 81}


Оценим информативность признаков:

In [43]:
best_model_trees = search.best_estimator_.trees

feature = 0

for tree in best_model_trees:
  feature += tree.feature_importances_

feature /= search.best_estimator_.n_trees

importance = pd.DataFrame(data={
    'Attribute': X.columns,
    'Importance': feature
})

importance = importance.sort_values(by='Importance', ascending=False)

importance.head()

,Attribute,Importance
3,Age,0.416691
6,NumOfProducts,0.304932
8,IsActiveMember,0.097274
1,Geography,0.069142
5,Balance,0.060125


**Оценим информативность признаков**  
Из полученной сводки рассматрим наиболее значимые индикаторы оттока клиентов:

*   Age - наиболее взрослые клиенты с меньшей вероятностью покинут банк
*   NumOfProducts - клиенты с бОльшим количеством продуктов банка с меньшей вероятностью его покинут
*   IsActiveMember - активные кленты с меньшей вероятностью покинут банк 
*   Balance - клиенты с высоким остатком на счетах с меньшей вероятностью покинут банк
*   Geography - местоположение клиента также можнт повлиять на решение покинуть банк (например, из-за неудобства обслуживания)



